In [79]:
import pandas as pd
import shortuuid


In [12]:
df = pd.read_csv('dataset/cleaned_nur_data.csv', index_col=0)
df.head()

,name,location,rank,description,tuition_and_fees,in_state,undergrad_enrollment,state,year,state_full,region
index,,,,,,,,,,,
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...",45320,0,5402,NJ,0,New Jersey,Northeast
1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...",47074,0,6699,MA,0,Massachusetts,Northeast
2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...",52491,0,5844,IL,0,Illinois,Midwest
3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...",49480,0,5532,CT,0,Connecticut,Northeast
4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...",55056,0,6102,NY,0,New York,Northeast


In [74]:
df_nur = df.copy()
df_nur = df_nur.rename(columns={'state': 'state_id'})
df_nur['rank_id'] = df_nur['location'].str[0] + '_'  + df_nur['description'].str[2:4] + df_nur['name'].str[8]
# df_nur['school_id'] = df_nur['name'].str[1] + df_nur['location'].str[0] + '_' + df_nur['state_full'].str[0]
df_nur

,name,location,rank,description,tuition_and_fees,in_state,undergrad_enrollment,state_id,year,state_full,region,rank_id,school_id
index,,,,,,,,,,,,,
0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...",45320,0,5402,NJ,0,New Jersey,Northeast,P_ine,rP_N
1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...",47074,0,6699,MA,0,Massachusetts,Northeast,C_rvs,aC_M
2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...",52491,0,5844,IL,0,Illinois,Midwest,NaN,nC_I
3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...",49480,0,5532,CT,0,Connecticut,Northeast,N_lec,aN_C
4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...",55056,0,6102,NY,0,New York,Northeast,NaN,oN_N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,University of Massachusetts--Dartmouth,"North Dartmouth, MA",220,"Located about 60 miles south of Boston, the Un...",19270,12588,7295,MA,0,Massachusetts,Northeast,N_cas,nN_M
227,University of Missouri--St. Louis,"St. Louis, MO",220,Undergraduates at University of Missouri--St. ...,26277,10065,13569,MO,0,Missouri,Midwest,NaN,nS_M
228,University of North Carolina--Greensboro,"Greensboro, NC",220,University of North Carolina--Greensboro is lo...,21595,6733,15951,NC,0,North Carolina,South,G_ivr,nG_N


In [76]:
import pandas as pd
import hashlib

# Sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C'],
    'column3': ['X', 'Y', 'Z']
}
df = pd.DataFrame(data)

# Function to generate a unique ID using SHA-256
def generate_unique_id(row):
    # Concatenate the values from multiple columns into a string
    concat_str = str(row['column1']) + str(row['column2']) + str(row['column3'])
    # Hash the concatenated string using SHA-256
    unique_id = hashlib.sha256(concat_str.encode()).hexdigest()
    return unique_id

# Apply the function to each row and create a new column 'unique_id'
df['unique_id'] = df.apply(generate_unique_id, axis=1)

print(df)


   column1 column2 column3                                          unique_id
0        1       A       X  c8bb4334a4a3c611f53c2379db031cf2e847ae3aafba30...
1        2       B       Y  ef8b44aaa6c2da5539720096e984f8ca25fabf6f9fe1f0...
2        3       C       Z  a81a2158f840239603ba795efb204dbca3fe889f58e9b8...


In [78]:
import pandas as pd
import shortuuid

# Sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C'],
    'column3': ['X', 'Y', 'Z']
}
df = pd.DataFrame(data)

# Function to generate a short unique ID using shortuuid
def generate_short_unique_id(row):
    # Concatenate the values from multiple columns into a string
    concat_str = str(row['column1']) + str(row['column2']) + str(row['column3'])
    # Generate a UUID from the concatenated string
    unique_id = shortuuid.uuid(name=concat_str)
    return unique_id

# Apply the function to each row and create a new column 'short_unique_id'
df['short_unique_id'] = df.apply(generate_short_unique_id, axis=1)

print(df)


   column1 column2 column3         short_unique_id
0        1       A       X  JoNYudakPHVcVZx5yGr9Ax
1        2       B       Y  WcbatQqoVTMQiTeWiyKwXr
2        3       C       Z  AYvwrLMjSUckRXySUujMnQ


In [42]:
df_nur['school_id'].nunique()

187